In [1]:
from anaconda_ai import AnacondaAIClient
from anaconda_ai.integrations.llama_index import AnacondaModel, AnacondaEmbeddingModel
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter
from llama_index.core.settings import Settings
from llama_index.vector_stores.postgres import PGVectorStore

In [2]:
llm = AnacondaModel("OpenHermes-2.5-Mistral-7B/q4_k_m")
embedding = AnacondaEmbeddingModel("bge-small-en-v1.5/q4_k_m")

Settings.llm = llm
Settings.embed_model = embedding

✓ OpenHermes-2.5-Mistral-7B_Q4_K_M.gguf (running)

✓ bge-small-en-v1.5_Q4_K_M.gguf (running)

In [3]:
documents = SimpleDirectoryReader(input_files=["../../README.md"]).load_data()

text_splitter = TokenTextSplitter(chunk_size=96, chunk_overlap=5)

In [4]:
client = AnacondaAIClient()

db = client.vector_db.create()

vector_store = PGVectorStore(
    connection_string=db.uri,
    async_connection_string=db.uri,
    table_name="readme",
    embed_dim=384,
    use_jsonb=True,
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
    show_progress=True,
)
query_engine = index.as_query_engine()

# Optionally use in-memory store rather than postgres
# vectorstore = VectorStoreIndex.from_documents(
#     documents, transformations=[text_splitter]
# )
# query_engine = vectorstore.as_query_engine()

Output()

✓ pg vector database started

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/46 [00:00<?, ?it/s]

In [5]:
query_engine.query("what is the backend that powers anaconda-ai").response

'The backend service for anaconda-ai is the Anaconda AI Navigator application. This package utilizes the backend API to list and download models and manage running tasks.'